In [1]:
import duckdb

In [2]:
duckdb.install_extension('spatial')
duckdb.install_extension('httpfs')

In [7]:
duckdb.load_extension('spatial')
duckdb.load_extension('httpfs')

In [9]:
duckdb.sql("SELECT ST_Buffer(ST_geomfromwkb(geometry), 100) as buffer FROM 'https://github.com/bertt/geoparquet/raw/main/geoparquet.tests/testfixtures/utrecht_kunstwerken_wkb.parquet'")

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                        buffer                                                        │
│                                                       geometry                                                       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((105.13098596953034 52.08975894165677, 103.2095140098534 32.58072674004394, 97.51893922065902 13.82141570…  │
│ POLYGON ((105.1400864426073 52.09465637512603, 103.21861448293035 32.5856241735132, 97.52803969373598 13.826313138…  │
│ POLYGON ((105.13237889801621 52.09068403548702, 103.21090693833926 32.5816518338742, 97.52033214914489 13.82234079…  │
│ POLYGON ((105.14165718270897 52.092077130964626, 103.22018522303202 32.5830449293518, 97.52961043383765 13.8237338…  │
│ POLYGON ((105.12305945158005 5

In [12]:
# Read Zone19_2013_12.gdb using the OpenFileGDB driver
# ST_Read(VARCHAR, sequential_layer_scan : BOOLEAN, spatial_filter : WKB_BLOB, open_options : VARCHAR[], layer : VARCHAR, allowed_drivers : VARCHAR[], sibling_files : VARCHAR[], spatial_filter_box : BOX_2D
duckdb.sql("CREATE TABLE AIS AS SELECT * FROM ST_Read('Zone19_2013_12.gdb')")


CatalogException: Catalog Error: Table with name "AIS" already exists!

In [13]:
duckdb.sql("SELECT * FROM AIS LIMIT 10")

┌───────┬───────┬─────────┬───────┬───┬──────────┬───────────┬──────────────┬────────────┬──────────────────────┐
│  SOG  │  COG  │ Heading │  ROT  │ … │ VoyageID │   MMSI    │ ReceiverType │ ReceiverID │        Shape         │
│ float │ float │  float  │ float │   │  int32   │   int32   │   varchar    │  varchar   │       wkb_blob       │
├───────┼───────┼─────────┼───────┼───┼──────────┼───────────┼──────────────┼────────────┼──────────────────────┤
│   0.4 │ 180.0 │   210.0 │   0.0 │ … │        1 │ 367901377 │ r            │ 003669930  │ POINT (-70.9775 42…  │
│   8.0 │ 108.4 │   102.0 │   0.0 │ … │        2 │ 366033886 │ r            │ 01RWAR1    │ POINT (-71.572608 …  │
│   0.0 │ 225.4 │   511.0 │ 128.0 │ … │        3 │ 367097606 │ r            │ 07CSJU1    │ POINT (-66.090852 …  │
│  12.9 │ 265.9 │   266.0 │   0.0 │ … │        4 │ 538035900 │ r            │ 003669930  │ POINT (-71.997133 …  │
│  14.0 │ 267.0 │   265.0 │   0.0 │ … │        5 │ 371060038 │ D            │ 01MN      

In [16]:
# Save the gdb as a parquet file
duckdb.sql("CREATE TABLE AIS_parquet AS SELECT * FROM AIS")

# Reference COPY <table> TO 'some/file/path/filename.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES');
duckdb.sql("COPY AIS_parquet TO 'AIS_parquet.parquet' WITH (FORMAT PARQUET)")

In [17]:
ais_from_parquet = duckdb.read_parquet('AIS_parquet.parquet')

In [18]:
len(ais_from_parquet)

4206986

In [ ]:
hgfjhg